In [429]:
#!unzip "/content/drive/MyDrive/train_dataset_Самолет2.zip" -d "/content/"

# Подгружаем библиотеки


In [430]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [431]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [432]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import KFold, TimeSeriesSplit
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error

import shap

import random, os


# Ставим сид

In [433]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(0)

In [434]:
#os.chdir(r'D:\!!ufa\cp_s2')

In [435]:
pd.options.display.max_columns=None

In [436]:
test = pd.read_excel('test.xlsx')
train = pd.read_excel('train.xlsx')

# Соединяем таблички

In [437]:
material_prices = pd.read_excel('Цены на сырье.xlsx')
train=train.merge(material_prices, on='dt', how='left')
test=test.merge(material_prices, on='dt', how='left')

In [438]:
market_material = pd.read_excel('Данные рынка стройматериалов.xlsx')
train=train.merge(market_material, on='dt', how='left')
test=test.merge(market_material, on='dt', how='left')

In [439]:
train.head(3)

,dt,Цена на арматуру,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т","Лом_3А, РФ CPT авто Центральный ФО, руб./т, без НДС","Лом_3А, РФ CPT авто Уральский ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, FOB США Восток, $/т","Лом_3А, FOB РФ Черное море, $/т","Чугун_FOB Бразилия Север, $/т","Лом_3А, РФ CPT авто Южный ФО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR ЮВА, $/т","Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС","Лом_Б26 легированный (Ni 9,3%), РФ CPT, руб./т, без НДС","ГБЖ_Россия, FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС","Чугун_FOB Бразилия Юг, $/т","ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,78-8,51, сухой вес, DAP Забайкальск-Манжули, $/т","Концентрат коксующегося угля_Россия марка К и смеси марок К, КО FCA руб./т, без НДС","Чугун_FOB Россия Дальневосточные порты, $/т","Кокс_Россия Урал Сибирь +25, FCA руб./т., без НДС","Лом_3А, РФ FCA ж/д Московский регион, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж+ГЖ FCA руб./т, без НДС","Чугун_FOB Россия Черное море, $/т","Лом_3А, РФ FCA ж/д Кемеровская обл, руб./т, без НДС","ЖРС_Россия аглоруда Fe 52-60%, FCA руб./т, без НДС","ЖРС_Россия концентрат Fe 64-68%, FCA руб./т, без НДС","Чугун_Россия, FCA руб./т, без НДС","Концентрат коксующегося угля_HCC Австралия, $/t FOB","ЖРС_Россия окатыши Fe 62-65,5% FCA руб./т, без НДС","ГБЖ_CFR Италия, $/т","Лом_3А, РФ CPT ж/д Уральский ФО, руб./т, без НДС","Концентрат коксующегося угля_Россия марка Ж FCA руб./т, без НДС","Чугун_CFR Западная Европа, $/т","Лом_3А, РФ FCA ж/д Челябинская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д ХМАО, руб./т, без НДС","Лом_HMS 1/2 80:20, CFR Турция, $/т","Чугун_CFR Китай, $/т","Лом_3А, РФ FCA ж/д Самарская обл, руб./т, без НДС","Концентрат коксующегося угля_Средняя шихта Россия FCA руб./т, без НДС","Концентрат коксующегося угля_Россия марка ГЖ FCA руб./т, без НДС","Лом_3А, РФ FCA ж/д Нижегородская обл, руб./т, без НДС","Чугун_CFR NOLA США, $/т","Лом_3А, РФ FCA ж/д Новосибирская обл, руб./т, без НДС","Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т",Импорт цемента ЖД,Экспорт цемента ЖД,Отгрузка цемента хоппер ЖД,Отгрузка цемента остальные ЖД,Производство цемента,Импорт цемента,Экспорт цемента,Потребление цемента,Отгрузка цемента,Цена цемента с НДС,Цена цемента без НДС,Производство бетона РФ,Производство бетона МО,Производство бетона СПБ
0,2015-01-05,32900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-12,32500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [440]:
metal_market = pd.read_excel('Показатели рынка металла.xlsx')
train=train.merge(metal_market, on='dt', how='left')
test=test.merge(metal_market, on='dt', how='left')

In [441]:
fuel = pd.read_excel('Топливо.xlsx')
train=train.merge(fuel, on='dt', how='left')
test=test.merge(fuel, on='dt', how='left')

In [442]:
Cargo = pd.read_excel('Грузоперевозки.xlsx')
train=train.merge(Cargo, on='dt', how='left')
test=test.merge(Cargo, on='dt', how='left')

In [443]:
index_LME = pd.read_excel('Индекс LME.xlsx', nrows=516).rename(columns={'дата': 'dt'})
train=train.merge(index_LME, on='dt', how='left')
test=test.merge(index_LME, on='dt', how='left')

In [444]:
macro_indicators = pd.read_excel('Макропоказатели.xlsx')
train=train.merge(macro_indicators, on='dt', how='left')
test=test.merge(macro_indicators, on='dt', how='left')

# Избавляемся от колонок с большим количеством пропусков, заполняем оставшиеся пустые данные

In [445]:
use_col = list(train.isna().sum()[(train.isna().sum()/len(train))<0.2].keys())
#use_col = set(use_col+list(test.isna().sum()[(test.isna().sum()/len(test))<0.2].keys()))

train = train[use_col].fillna(method='ffill').fillna(method='bfill')
test = test[use_col].fillna(method='ffill').fillna(method='bfill')

# Разделяем дату на временные фичи

In [446]:
pref_days = [0]
for year in range(2015 ,2023):
    pref_days.append(pd.Timestamp(f'{year}-12-31').dayofyear)
print(pref_days)
for i in range(1, len(pref_days)):
    pref_days[i] += pref_days[i-1]
print(pref_days)


def prepare(x):
    #if x=='nan':
    #    return -1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
    x = x.split(' ')
    #print(x)
    date = x[0].split('-')
    #time = x[1].split(':')
    year,month,day = map(int, date)
    timestamp = pd.Timestamp(f'{year}-{month}-{day}')
    #hour = 0 #int(time[0])
    dayofyear = timestamp.dayofyear
    dayofweek = timestamp.dayofweek
    ind_date = dayofyear + pref_days[year - 2015] - 1
    #ind_hour = ind_date * 24 + hour
    if dayofweek >= 5:
        days_to_weekend = 0
    else:
        days_to_weekend = min(dayofweek + 1, abs(5 - dayofweek))
    return year, month, ind_date
print(test.columns)


add_columns = list(map(lambda x: 'dt'+'_'+x, ['year', 'month', 'ind_date']))

train[add_columns] = list(train['dt'].astype(str).apply(prepare))
test[add_columns] = list(test['dt'].astype(str).apply(prepare))

for col in add_columns:

    train[col] = train[col].astype(np.int32)
    test[col] = test[col].astype(np.int32)

del train['dt'],test['dt']

[0, 365, 366, 365, 365, 365, 366, 365, 365]
[0, 365, 731, 1096, 1461, 1826, 2192, 2557, 2922]
Index(['dt', 'Цена на арматуру',
       'Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС',
       'Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т',
       'Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС',
       'Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС',
       'Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС',
       'ЖРС_Китай Iron ore fines Fe 62%, CFR',
       'Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС',
       'ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т',
       'Лом_HMS 1/2 80:20, FOB США Восток, $/т',
       'Лом_3А, FOB РФ Черное море, $/т', 'Чугун_FOB Бразилия Север, $/т',
       'Лом_HMS 1/2 80:20, CFR ЮВА, $/т',
       'Концентрат коксующегося угля_Россия марка КС FCA руб./т, без НДС',
       'Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС',
       'ЖРС_Российский концентрат Fe 62,5-67,5%, SiO2 0,7

# Избавляемся от колонок с высокой корреляцией и которые ухудшали валидацию

In [447]:
to_drop=['Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т', 'Лом_HMS 1/2 80:20, CFR Турция, $/т',
         'Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС',
         'Лом_3А, РФ FCA ж/д респ. Татарстан, руб./т, без НДС',
         'Чугун_Россия, FCA руб./т, без НДС',
         #'ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т',
         'dt_ind_date',
         'Чугун_FOB Бразилия Север, $/т',
         'Кокс_Россия Урал Сибирь +25, FCA руб./т., без НДС',
         'ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС',
         'dt_month',
         'Чугун_FOB Россия Черное море, $/т'
         #'Концентрат коксующегося угля_Россия марка ГЖ FCA руб./т, без НДС'
        ]
train=train.drop(to_drop, axis=1)
test=test.drop(to_drop, axis=1)

# Соединяем строки данных и достаем идущие после них цены

Пример [[0,0,0],
        [1,2,3],
        [4,5,6]] --> [[0,0,0,1,2,3,4,5,6]]


Для строки x таргетом будет цена строки x+1, x+2, ...



In [448]:
def advanced_reshape(x, portion_size: int):
    for i in range(x.shape[0] - portion_size + 1):
        yield x[i:i + portion_size, :].reshape(1, -1)

In [449]:
def form_portioned_array(x, portion_size: int):
    return np.concatenate(list(advanced_reshape(x, portion_size)),
                          axis=0)

In [450]:
from typing import Tuple


def form_lags(x: pd.Series, lags: Tuple[int]) -> pd.DataFrame:
    x_lags = tuple()
    for lag in lags:
        x_lags += (x.shift(lag), )
    return pd.concat(x_lags, axis=1).values


# Генерация фич и скейлер

In [451]:
def generate_feats(data, test, gen_ft=None, max_ft=100, ratio=0.85):
    numerics = ['float64', 'int64', 'uint8']
    fts = data.copy()
    corrs = fts.corr()
    target='Цена на арматуру'
    if gen_ft is None:
        k = len(fts.columns) #длина колонок
        gen_ft = []
        for i in tqdm(range(k)):
            #for j in range(i+1, k):

            try:
                t1 = fts[fts.columns[i]].dtypes
                t2 = fts[target].dtypes
                if t1 not in numerics or t2 not in numerics:
                    continue
                if corrs[fts.columns[i]][target] != np.nan and ((corrs[fts.columns[i]][target] >= ratio) or (corrs[fts.columns[i]][target] <= -ratio)) and fts.columns[i]!=target:
                    gen_ft.append([i, target])
            except:
                pass
    for q in tqdm(range(min(max_ft, len(gen_ft)))):
        i, target = gen_ft[q]
        #test[f"{test.columns[i]}*{target}"] = test[test.columns[i]] * test[target]
        #test[f"{test.columns[i]}/{target}"] = test[test.columns[i]] / (test[target]+1e-9)
        #test[f"{target}/{test.columns[i]}"] = test[target] / (test[test.columns[i]]+1e-9)
        #test[f"{test.columns[i]}_func1_{target}"] = test[test.columns[i]] * test[target] / (test[test.columns[i]] + test[target] + 1e-9) ** 2
        test[f"{test.columns[i]}_func2_{target}"] = abs(test[test.columns[i]] - test[target]) ** 0.5 * abs(test[test.columns[i]] + test[target]) ** 0.5

        #fts[f"{fts.columns[i]}*{target}"] = fts[fts.columns[i]] * fts[target]
        #fts[f"{fts.columns[i]}/{target}"] = fts[fts.columns[i]] / (fts[target]+1e-9)
        #fts[f"{target}/{fts.columns[i]}"] = fts[target] / (fts[fts.columns[i]]+1e-9)
        #fts[f"{fts.columns[i]}_func1_{target}"] = fts[fts.columns[i]] * fts[target] / (fts[fts.columns[i]] + fts[target] + 1e-9) ** 2
        fts[f"{fts.columns[i]}_func2_{target}"] = abs(fts[fts.columns[i]] - fts[target]) ** 0.5 * abs(fts[fts.columns[i]] + fts[target]) ** 0.5

    # fts = reduce_mem_usage(fts)
    return fts, test

In [452]:
#train.corr()

In [453]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols=train.columns

scaler=scaler.fit(pd.concat([train, test], axis=0))

train=pd.DataFrame(scaler.transform(train), columns=cols[0:])
test=pd.DataFrame(scaler.transform(test), columns=cols[0:])

In [454]:
train, test=generate_feats(train, test)

100%|██████████| 10/10 [00:00<00:00, 219.67it/s]


In [455]:
portion_size = 1

train_data = form_portioned_array(train.values, portion_size)[:-3]
test_data = form_portioned_array(test.values, portion_size)[:-3]

In [456]:
train_targets = form_lags(train['Цена на арматуру'], lags=(-portion_size,-portion_size-1,-portion_size-2))[:-3]
test_targets = form_lags(test['Цена на арматуру'], lags=(-portion_size,-portion_size-1,-portion_size-2))[:-3]

# CV для временных данных

для каждого таргета (как далеко идет предсказание обучаем отдельно кб, т.е модель, которая предсказывает на неделю вперед и модель, которая предсказывает на 2 недели вперед - это разные модели)

In [457]:
from catboost import CatBoostRegressor

In [ ]:
X=train_data
y=train_targets

splits=14
n_models=3

models=[]
best_scores=[1e9 for i in range(n_models)]
real_scores=[1e30 for i in range(n_models)]
for i in range(n_models):
    cb=CatBoostRegressor(iterations=2000,verbose=1000, random_state=0)
    models.append(cb)
best_models=models.copy()

kf = TimeSeriesSplit(n_splits=splits)

for i, (train_index, val_index) in enumerate(kf.split(X,y)):

    print('FOLD ',i)

    X_train, X_val = X[train_index], X[val_index]

    for j, model in enumerate(models):

        print('MODEL ', j)

        y_train, y_val = y[:,j][train_index], np.array(y[:,j][val_index])

        model.fit(X_train, y_train, eval_set=(X_val, y_val))

        pred = np.array(model.predict(X_val))

        score = mean_absolute_error(pred, y_val)

        pred = (pred*scaler.scale_[0]) + scaler.mean_[0]#2**pred
        y_val = (y_val*scaler.scale_[0]) + scaler.mean_[0]#2**y_valA

        real_score = mean_absolute_error(pred, y_val)

        print(score,real_score)

        if score<best_scores[j]:
            best_scores[j] = score
            real_scores[j] = real_score
            best_models[j] = model

        models[j]=best_models[j]

FOLD  0
MODEL  0
Learning rate set to 0.019529
0:	learn: 0.2009009	test: 0.3025521	best: 0.3025521 (0)	total: 460us	remaining: 920ms
1000:	learn: 0.0020245	test: 0.2069750	best: 0.2069750 (1000)	total: 3.38s	remaining: 3.38s
1999:	learn: 0.0018883	test: 0.2066746	best: 0.2066740 (1841)	total: 7.25s	remaining: 0us

bestTest = 0.2066740406
bestIteration = 1841

Shrink model to first 1842 iterations.
0.1751480642710635 2283.4676104243654
MODEL  1
Learning rate set to 0.019529
0:	learn: 0.1867704	test: 0.2882834	best: 0.2882834 (0)	total: 677us	remaining: 1.35s
1000:	learn: 0.0066629	test: 0.2095917	best: 0.2095739 (905)	total: 2.83s	remaining: 2.83s
1999:	learn: 0.0066090	test: 0.2095533	best: 0.2095296 (1513)	total: 5.86s	remaining: 0us

bestTest = 0.2095296439
bestIteration = 1513

Shrink model to first 1514 iterations.
0.1778721637181411 2318.9826638214154
MODEL  2
Learning rate set to 0.019529
0:	learn: 0.1729660	test: 0.2720653	best: 0.2720653 (0)	total: 490us	remaining: 981ms
1000:	

In [ ]:
print(best_scores)#1324 1583 2001
print(real_scores)#1500 1700 2100

# Сохраняем данные и модели для прогонки теста

In [ ]:
import pickle

In [ ]:
for i,model in enumerate(best_models):
  with open(f'model{i}.pkl','wb') as f:
    pickle.dump(model,f)

np.savetxt('test_data.txt',test_data)